In [1]:
# imports

#standard
import pandas as pd
import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [55]:
#Custom Functions 

def d6():
    #Return a random integer between [1-6]    
    roll = random.randint(1,6)
    return(roll)

def roll_four_drop_lowest():
    single_stat_list = []
    for x in range(0,4):
        roll = d6()
        single_stat_list.append(roll)
    single_stat_list.sort()
    single_stat_list = single_stat_list[1:4] #Removes the lowest
    return(sum(single_stat_list))

def colville_calculation():
    stat_list = []
    
    while True:
        stat_list = [roll_four_drop_lowest() for i in range(6)]
        
        # Count how many stats are 15 or higher
        high_stats = [stat for stat in stat_list if stat >= 15]
        
        if len(high_stats) >= 2:
            break  # Breaks loop if at least 2 stats are >= 15
    
    return stat_list

def mercer_calculation():
    stat_list = []
    
    while True:
        stat_list = [roll_four_drop_lowest() for i in range(6)]
        
        # Only 
        if sum(stat_list) >= 70:
            break  # Breaks loop if stat total isn't about 70
    
    return stat_list

In [56]:
x = colville_calculation()
y = mercer_calculation()

print(x)
print(y)

[15, 17, 14, 18, 9, 8]
[13, 14, 14, 14, 12, 12]
